In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

In [2]:
import pyspark
from pyspark.sql import SQLContext, Row
from pyspark import SparkContext
import numpy as np
sc = SparkContext('local[*]')

In [3]:
import plotly
from plotly.graph_objs import Scatter, Layout, Bar, Figure, Layout

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

init_notebook_mode(connected=True)

In [4]:
filenames = [
            'orchestra_log_2_channel.csv', 
            'orchestra_log_3_channel.csv', 
            'orchestra_log_4_channel.csv', 
            'orchestra_log_5_channel.csv'
           ]
# filenames = [
#             'rlm_log_2_channel.csv', 
#             'rlm_log_3_channel.csv', 
# #             'orchestra_log_4_channel.csv', 
#             'rlm_log_5_channel.csv'
#            ]
means2 = []
stdevs2 = []
means3 = []
stdevs3 = []

for filename in filenames:
    tf = sc.textFile(filename)
    rdd = tf.map(lambda x: (x.split(";")[0], x)).groupByKey().mapValues(list)
    hopping2 = rdd.filter(lambda x: len(x[1]) == 2)
    hopping3 = rdd.filter(lambda x: len(x[1]) == 3)

    latency2 = hopping2.map(lambda x: int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
    latency3 = hopping3.map(lambda x: int(x[1][2].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
    
    means2.append(np.mean(latency2))
    means3.append(np.mean(latency3))
    stdevs2.append(np.std(latency2))
    stdevs3.append(np.std(latency3))
    

/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning:

Mean of empty slice.

/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning:

Degrees of freedom <= 0 for slice



In [5]:
print 'means2  : ', means2
print 'means3  : ', means2
print 'stdevs2 : ', stdevs2
print 'stdevs3 : ', stdevs3

means2  :  [504247.84507042251, 291426.39130434784, 521261.04918032786, 319624.04166666669]
means3  :  [504247.84507042251, 291426.39130434784, 521261.04918032786, 319624.04166666669]
stdevs2 :  [962496.63522418472, 319593.51447577483, 889209.25133147906, 402872.74823597417]
stdevs3 :  [1408578.8025759957, 312171.4058159542, 894186.72768027452, nan]


In [6]:
trace1 = Bar(
    x = filenames,
    y = means2,
    name='1 hopping',
    error_y=dict(
        type = 'data',
        array = stdevs2,
        visible = True
    )
)
trace2 = Bar(
    x = filenames,
    y = means3,
    name='2 hopping',
    error_y=dict(
        type = 'data',
        array = stdevs3,
        visible = True
    )
)

data = [trace1, trace2]
layout = Layout(
    barmode='group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='error-bar-bar')